### Compute the mean and std of images in the datasets

In [80]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

NUM_WORKERS = os.cpu_count() // 1

# data_path = './Bottle images'
# transform_img = transforms.Compose(
#     [transforms.CenterCrop(150),
#      transforms.ToTensor()]
# )

# image_data = ImageFolder(
#     root=data_path, transform=transform_img
# )

# image_data_loader = DataLoader(
#     image_data,
#     # batch size is whole datset
#     batch_size=len(image_data),
#     shuffle=False,
#     num_workers=NUM_WORKERS)


# def mean_std(loader):
#     images, lebels = next(iter(loader))
#     # shape of images = [b,c,w,h]
#     mean, std = images.mean([0, 2, 3]), images.std([0, 2, 3])
#     return mean, std


# IMAGES_MEAN, IMAGES_STD = mean_std(image_data_loader)
IMAGES_MEAN, IMAGES_STD = torch.tensor([0.4211, 0.4920, 0.4686]), torch.tensor([0.1437, 0.1570, 0.1546])


### Create Lightning Data Module (subclass)

In [81]:
import os
import torchmetrics
from torch import nn, optim
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision.models as models
from pytorch_lightning import Trainer
from torch.utils.data import random_split
from pytorch_lightning.callbacks import ModelCheckpoint


os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
pl.seed_everything(42)


class BottleDataset(pl.LightningDataModule):
    def __init__(self, data_dir: str = None, batch_size: int = int(2**8), num_workers=NUM_WORKERS):
        super().__init__()
        self.data_dir = data_dir or os.getcwd()
        self.num_workers = num_workers
        self.batch_size = batch_size
        self.train_transform = transforms.Compose(
            [transforms.CenterCrop(150),
             transforms.RandomHorizontalFlip(p=0.5),
             transforms.RandomVerticalFlip(p=0.5),
             transforms.ToTensor(),
             transforms.Normalize((IMAGES_MEAN), (IMAGES_STD))]
        )
        self.test_transform = transforms.Compose(
            [transforms.Grayscale(num_output_channels=1),
             transforms.ToTensor(),
             transforms.Normalize((IMAGES_MEAN), (IMAGES_STD))]
        )

    def prepare_data(self):
        self.data = ImageFolder(
            self.data_dir, transform=self.train_transform)

    def setup(self, train_ratio: float = 0.9, stage=None):
        if stage == 'fit' or stage is None:
            train_amount = int(len(self.data) * train_ratio)
            self.train, self.test = random_split(
                self.data, [train_amount, len(self.data) - train_amount])
            train_amount = int(len(self.train) * train_ratio)
            self.train, self.val = random_split(
                self.train, [train_amount, len(self.train) - train_amount])
        if stage == 'test' or stage is None:
            pass

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True)


Global seed set to 42


### Create Lightning Module subclass (model)

In [82]:
'''
We inherit a Resnet model
https://github.com/Stevellen/ResNet-Lightning/blob/master/resnet_classifier.py
'''


class ResNetClassifier(pl.LightningModule):
    def __init__(self, num_classes=8, resnet_version=18, transfer=False):
        super().__init__()
        self.__dict__.update(locals())
        resnets = {
            18: models.resnet18, 34: models.resnet34,
            50: models.resnet50, 101: models.resnet101,
            152: models.resnet152
        }
        self.acc = torchmetrics.Accuracy()
        self.lr = 1e-3
        # instantiate loss criterion
        self.loss = nn.BCEWithLogitsLoss() if num_classes == 2 else nn.CrossEntropyLoss()
        # Do not use a pretrained ResNet backbone
        self.resnet_model = resnets[resnet_version](pretrained=transfer)
        # Replace old FC layer with Identity so we can train our own
        linear_size = list(self.resnet_model.children())[-1].in_features
        # replace final layer for fine tuning
        self.resnet_model.fc = nn.Linear(linear_size, num_classes)

    def forward(self, X):
        return self.resnet_model(X)

    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=self.lr)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        if self.num_classes == 2:
            y = F.one_hot(y, num_classes=2).float()
        loss = self.loss(preds, y)
        acc = self.acc(preds, y)
        # Logging the loss
        self.log("train_loss", loss, on_epoch=True, on_step=True, logger=True)
        self.log('train_acc', acc, on_epoch=True, on_step=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        if self.num_classes == 2:
            y = F.one_hot(y, num_classes=2).float()
        loss = self.loss(preds, y)
        acc = self.acc(preds, y)
        # Logging the loss
        self.log("val_loss", loss, on_epoch=True, on_step=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, on_step=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        if self.num_classes == 2:
            y = F.one_hot(y, num_classes=2).float()
        loss = self.loss(preds, y)
        acc = self.acc(preds, y)
        # perform logging
        self.log("test_loss", loss, on_epoch=True, on_step=True, logger=True)
        self.log("test_acc", acc, on_epoch=True, on_step=True, logger=True)


In [83]:
def find_best_ckpt(dir):
    check_points = sorted([(ckpt, ckpt.rsplit('=', 1)[1].rsplit('.', 1)[0]) for ckpt in os.listdir(dir)], key = lambda x: float(x[1]))
    return check_points[0][0]

In [84]:
data_path = "Bottle images/"
model_cp_path = "Model Checkpoints"
model_cp_filename = "Bottle_model"
data_module = BottleDataset(data_path)

checkpoint_params = {'filename': '{epoch}-{step}-{val_loss:.3f}', 'save_top_k': 1, 'monitor':"val_acc", 'mode':'max', 'every_n_epochs':5, 'save_on_train_epoch_end':True}
trainer = pl.Trainer(
    max_epochs=100,
    auto_lr_find=True,
    gpus=-1,
    auto_scale_batch_size=True,
    check_val_every_n_epoch=5,
    detect_anomaly=True,
    precision=16,
    callbacks=ModelCheckpoint(dirpath=model_cp_path, **checkpoint_params),
)
model = ResNetClassifier()
if os.listdir(model_cp_path):
    model = ResNetClassifier.load_from_checkpoint(f"{model_cp_path}/epoch=49-step=199-val_loss=0.675.ckpt")
trainer.fit(model, data_module)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | acc          | Accuracy         | 0     
1 | loss         | CrossEntropyLoss | 0     
2 | resnet_model | ResNet           | 11.2 M
--------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
22.361    Total estimated model params size (MB)


Global seed set to 42


Epoch 99: 100%|██████████| 5/5 [00:24<00:00,  4.91s/it, loss=0.191, v_num=12]


In [85]:
import webbrowser

# # Open a url after done
webbrowser.open('https://youtu.be/5dwxGvmUG90?t=53')

True